
# Projeto ICMC Júnior
## Objetivo: Prever se um funcionário vai sair da empresa (attrition).

### Etapa final: Modelagem dos dados

In [17]:
import numpy as np
import pandas as pd

In [18]:
# Leitura do dataset (usamos, agora, a base limpa gerada na etapa 1 do projeto)
df = pd.read_csv('dados_limpos.csv')

#### Adequação e preparação final dos dados

In [19]:
# Para preparar os dados para a aplicação de um modelo de Machine Learning, devemos garantir que todos os valores sejam numéricos.
# Assim, convertemos as colunas qualitativas (categóricas) em quantitativas (numéricas).
# Para isso, usamos Label Encoding para colunas binárias e One-Hot Encoding para colunas nominais com mais de duas categorias.

# Função para realizar Label Encoding em colunas qualitativas binárias
def encoding_binary_columns(column, map_values):
    if column not in df.columns:
        print(f"Erro, coluna {column} não encontrada no dataframe")
    
    print(f"Convertendo coluna: '{column}'...")
    print(f"  Valores ANTES: {df[column].unique()}")
    df[column] = df[column].map(map_values) # Aplica mapeamento na coluna
    print(f"  Valores DEPOIS: {df[column].unique()}\n")

encoding_binary_columns('Attrition', {'yes': 1, 'no': 0})
encoding_binary_columns('OverTime', {'yes': 1, 'no': 0})
encoding_binary_columns('Gender', {'female': 1, 'male': 0})

# Lista de colunas do tipo qualitativo nominal
colunas_nominais = [
    'BusinessTravel', 
    'Department', 
    'EducationField', 
    'JobRole', 
    'MaritalStatus'
]
# Realiza a conversão do dados de qualitativo para quantitativo (como consequência, teremos um aumento de colunas de 31 para 45).
df = pd.get_dummies(df, columns=colunas_nominais, dtype = int, drop_first=True) # 'drop_first=True' evita multicolinearidade


Convertendo coluna: 'Attrition'...
  Valores ANTES: ['yes' 'no']
  Valores DEPOIS: [1 0]

Convertendo coluna: 'OverTime'...
  Valores ANTES: ['yes' 'no']
  Valores DEPOIS: [1 0]

Convertendo coluna: 'Gender'...
  Valores ANTES: ['female' 'male']
  Valores DEPOIS: [1 0]



In [20]:
# Com os dados preparados, é feita uma divisão de treino e teste.
from sklearn.model_selection import train_test_split

# Definição da variável alvo (target) e das variáveis preditoras (features)
X = df.drop('Attrition', axis=1) # Variáveis preditoras (todas colunas menos 'Attrition')
y = df['Attrition'] # Variável alvo
print(f"Dimensão das variáveis preditoras (X): {X.shape}")
print(f"Dimensão da variável alvo (y): {y.shape}\n")

# Divisão dos dados em conjuntos de treino e teste (75% treino, 25% teste)
# 'random_state=42' garante reprodutibilidade dos resultados; 'stratify=y' mantém a proporção original das classes na divisão
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
print(f"Tamanho do conjunto de treino: {X_treino.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_teste.shape[0]} amostras\n")


Dimensão das variáveis preditoras (X): (1470, 44)
Dimensão da variável alvo (y): (1470,)

Tamanho do conjunto de treino: 1102 amostras
Tamanho do conjunto de teste: 368 amostras



In [21]:
# Tendo dividido os dados, aplica-se uma padronização sobre os valores (nem todos os modelos de ML exigem isso, mas é uma boa prática).
# Nisso, a média de todos os valores se torna 0 e o desvio padrão, 1.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Ajusta o scaler apenas com os dados de treino para evitar vazamento de dados (data leakage)
scaler.fit(X_treino)
# Aplica a padronização nos conjuntos de treino e teste
X_treino_padronizado = scaler.transform(X_treino)
X_teste_padronizado = scaler.transform(X_teste)
print(f"\nMédia da primeira coluna de X_treino_padronizado (deve ser ~0): {X_treino_padronizado[:, 0].mean()}")
print(f"\nDesvio Padrão da primeira coluna de X_treino_padronizado (deve ser ~1): {X_treino_padronizado[:, 0].std()}")





Média da primeira coluna de X_treino_padronizado (deve ser ~0): -1.6119390557171058e-17

Desvio Padrão da primeira coluna de X_treino_padronizado (deve ser ~1): 1.0


#### Aplicação do modelo de Regressão Logística

In [22]:
# Por fim, com todos os dados preparados e separados devidamente, podemos aplicar o modelo.
# Escolhemos a Regressão Logística, resumidamente, por ser um modelo simples, eficiente e interpretável.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cria um objeto do modelo. 'random_state=42' garante reprodutibilidade dos resultados.
modelo = LogisticRegression(random_state=42, class_weight='balanced')

print("Treinando o modelo de Regressão Logística...")
modelo.fit(X_treino_padronizado, y_treino) # Uso dos dados padronizados de treino
print("Treinamento concluído!")

print("\nAvaliação do modelo nos dados de teste---")
# Cria previsões usando o modelo treinado
y_previsoes = modelo.predict(X_teste_padronizado) # Uso dos dados padronizados de teste

# Avaliação das previsões (comparar y_previsoes com y_teste)
acuracia = accuracy_score(y_teste, y_previsoes)
print(f"\nAcurácia (Accuracy): {acuracia * 100:.2f}%")

# Relatório de classificação.
print("\nRelatório de Classificação:")
print(classification_report(y_teste, y_previsoes))

# Matriz de confusão.
print("\nMatriz de Confusão:")
print(confusion_matrix(y_teste, y_previsoes))

Treinando o modelo de Regressão Logística...
Treinamento concluído!

Avaliação do modelo nos dados de teste---

Acurácia (Accuracy): 78.53%

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.92      0.81      0.86       309
           1       0.40      0.64      0.49        59

    accuracy                           0.79       368
   macro avg       0.66      0.73      0.68       368
weighted avg       0.84      0.79      0.80       368


Matriz de Confusão:
[[251  58]
 [ 21  38]]


In [ ]:
# Com os dados resultantes da Regressão Logística, é possível tentar um modelo mais complexo, como o Random Forest.
# O código segue a mesma lógica do trecho que usa Regressão Logística, mas aplicando o modelo Random Forest.

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cria um objeto do modelo Random Forest.
rf_modelo = RandomForestClassifier(
    n_estimators=100,        # Número de árvores na floresta
    random_state=42,         # Para reprodutibilidade
    class_weight='balanced',  # Foca na classe 1 (foca em prever "yes" corretamente)
    max_depth=8              # Limita a profundidade das árvores para evitar overfitting
)

print("Treinando o modelo Random Forest...")
rf_modelo.fit(X_treino, y_treino) # Obs.: para este modelo, não usamos os dados padronizados
print("Treinamento concluído!")

print("\nAvaliação do modelo nos dados de teste---")
y_previsoes_rf = rf_modelo.predict(X_teste) # Novamente, não usamos dados padronizados

print(f"\nAcurácia (Accuracy): {accuracy_score(y_teste, y_previsoes_rf) * 100:.2f}%")

print("\nRelatório de Classificação (Random Forest):")
print(classification_report(y_teste, y_previsoes_rf))

print("\nMatriz de Confusão (Random Forest):")
print(confusion_matrix(y_teste, y_previsoes_rf))

Treinando o modelo Random Forest...
Treinamento concluído!

Avaliação do modelo nos dados de teste---

Acurácia (Accuracy): 82.88%

Relatório de Classificação (Random Forest):
              precision    recall  f1-score   support

           0       0.86      0.96      0.90       309
           1       0.41      0.15      0.22        59

    accuracy                           0.83       368
   macro avg       0.63      0.56      0.56       368
weighted avg       0.78      0.83      0.79       368


Matriz de Confusão (Random Forest):
[[296  13]
 [ 50   9]]
